In [1]:
#import the needed libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import warnings
warnings.filterwarnings('ignore') #Supress unnecessary warnings for readability and cleaner presentation
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
from IPython.display import display
import pickle
color = sns.color_palette()

import pickle
pd.set_option('display.float_format', lambda x: '%.3f' % x) #Limiting floats output to 3 decimal points
import gc
print (gc.isenabled())

import tensorflow
import keras
from keras.models import Model,Sequential

from keras.layers.core import Dense, Reshape, Lambda,Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers import Input, Embedding, merge
from keras import backend as K
from subprocess import check_output
#print(check_output(['dir', 'input/']).decode("utf8")) #check the files available in the directory

True


Using TensorFlow backend.


In [2]:
# concatenate all product-ids into a single string
# thanks to https://www.kaggle.com/eoakley/start-here-simple-submission

def products_concat(series):
    out = ''
    for product in series:
           out = out + str(int(product)) + ' '
    
    if out != '':
        return out.rstrip()
    else:
        return 'None'

In [3]:
def df_split(df):
    df_reordered = df[df['reordered'] == 1]
    df_reordered = df_reordered.groupby('order_id')['product_id'].apply(products_concat)
    
    try:
        df_reordered = df_reordered.reset_index()
        df_reordered.columns = ['order_id','products_list']
    except:
        df_reordered = df_reordered.reset_index(level = ['order_id'])
        df_reordered.columns = ['order_id','products_list']
        
    return df_reordered

In [4]:
def multilabel_fscore(ytrue, ypred):
    """
    ex1:
    y_true = [1, 2, 3]
    y_pred = [2, 3]
    return: 0.8
    
    ex2:
    y_true = ["None"]
    y_pred = [2, "None"]
    return: 0.666
    
    ex3:
    y_true = [4, 5, 6, 7]
    y_pred = [2, 4, 8, 9]
    return: 0.25
    
    """
    #y_true = K.eval(ytrue)
    #y_pred = K.eval(ypred)
    F1_score = []
    y_pred = []
    y_true = []
    for items in range(len(ytrue)):
        y_true = ytrue[items]
        y_pred = ypred[items]
        correct = sum([1 for i in y_pred if i in y_true])
        if correct > 0:

            precision = correct / len(y_pred)

            recall =    correct / len(y_true)

            F1_score.append((2 * precision * recall) / (precision + recall))

        else:
            F1_score.append(0)

    return np.mean(F1_score)

#if __name__ != '__main__':

print(multilabel_fscore([[2,3],['None']], [[2,3,4],['None']]))

0.9


In [ ]:
#Now let's get and put the data in  pandas dataframe
order_products_train = pd.read_csv('input/order_products__train.csv',nrows=100000)
order_products_prior = pd.read_csv('input/order_products__prior.csv',nrows=100000)
orders = pd.read_csv('input/orders.csv',engine ='c')
#aisles = pd.read_csv('input/aisles.csv')
#departments = pd.read_csv('input/departments.csv')

In [ ]:
#create one hot encode of aisle and dept
products = pd.read_csv('input/products.csv')
value = list(products['product_id'].astype(str))
value.append('None')
df_additional = products[['product_id','aisle_id','department_id']]
df_additional[['aisle_id','department_id']] = df_additional[['aisle_id','department_id']].astype(str)
df_additional = pd.get_dummies(df_additional, columns=['aisle_id','department_id'])
display(df_additional.head(2))

In [ ]:
prior_reo = order_products_prior[order_products_prior['reordered'] == 1].drop('reordered', axis=1)
display(prior_reo.head(1))
prior_reo = prior_reo.merge(df_additional, on='product_id', how='left')
print('joined')
prior_reo = prior_reo.drop(['product_id','add_to_cart_order'], axis =1)
print('aggregating')
prior_reo = prior_reo.groupby('order_id').sum()
prior_reo = prior_reo.reset_index()
#print('saving')
#prior_reo.to_csv('input/prior_prods.csv', index=False)
display(prior_reo.head(2))

In [ ]:
train_reo = order_products_train[order_products_train['reordered'] == 1].drop('reordered', axis=1)
display(train_reo.head(1))
train_reo = train_reo.merge(df_additional, on='product_id', how='left')
train_reo = train_reo.drop(['product_id','add_to_cart_order'], axis =1)
print('aggregating')
train_reo = train_reo.groupby('order_id').sum()
train_reo = train_reo.reset_index()
#print('saving')
#train_reo.to_csv('input/train_prods.csv', index=False)
display(train_reo.head(2))

In [ ]:
#find the number of the last order placed
#split orders
test_orders  = orders[orders['eval_set'] == 'test' ]
prior_orders = orders[orders['eval_set'] == 'prior']
train_orders = orders[orders['eval_set'] == 'train']
prior_orders['num_orders'] = prior_orders.groupby(['user_id'])['order_number'].transform(max)
train_orders['num_orders'] = train_orders.groupby(['user_id'])['order_number'].transform(max)
test_orders['num_orders'] = test_orders.groupby(['user_id'])['order_number'].transform(max)

In [ ]:
df_prior_prod = df_split(order_products_prior)

df_train = prior_orders.merge(df_prior_prod, on='order_id', how='left')
df_train = df_train.merge(prior_reo, on='order_id', how='left')

display(df_train.tail(2))

In [ ]:
df_target_products = df_split(order_products_train)

df_eval = train_orders.merge(df_target_products, on='order_id', how='left')
df_eval = df_eval.merge(train_reo, on='order_id', how='left')
display(df_eval.head(2))

In [ ]:
df_train['products_list'] = df_train['products_list'].fillna('None')
df_eval['products_list'] = df_eval['products_list'].fillna('None')
display(df_train.head(2))

In [ ]:
del order_products_train,order_products_prior,orders,products,prior_orders,train_orders
del df_target_products,df_prior_prod, train_reo, prior_reo, df_additional
gc.collect()

In [ ]:
df_eval = df_eval.drop(['eval_set','user_id','order_number'], axis =1)
df_train = df_train.drop(['eval_set','user_id','order_number'], axis =1)
#display(df_eval[df_eval['products_list']== 'None'].head())

In [ ]:
#save data
pickle.dump(df_eval, open('eval_orders.p', 'wb'), protocol=4)

pickle.dump(df_train, open('train_orders.p', 'wb'), protocol=4)

pickle.dump(test_orders, open('test_orders.p', 'wb'), protocol=4)

## Checkpoint
data can be loaded from here

In [5]:
'''
evals = open('eval_orders.p','rb')
df_eval = pickle.load(evals)
evals.close()
'''
train = open('train_orders.p','rb')
df_train = pickle.load(train)
train.close()
'''
test = open('test_orders.p','rb')
test_orders = pickle.load(test)
test.close()
'''

"\ntest = open('test_orders.p','rb')\ntest_orders = pickle.load(test)\ntest.close()\n"

In [ ]:
X_train = df_train.drop(['products_list'], axis = 1)

In [ ]:
tot_comb = df_train.products_list.unique()
display(X_train.describe())

### setting up keras

In [ ]:
X_train = X_train.set_index('order_id')

In [ ]:
X_train[['order_dow', 'order_hour_of_day']] = X_train[['order_dow', 'order_hour_of_day']].astype(str)
X_train = pd.get_dummies(X_train, columns=['order_dow', 'order_hour_of_day'])

X_train = X_train.fillna(0)
keras_train = np.array(X_train)

display(X_train.head())

In [ ]:
#vectorise the products list
#from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

dicts = []
for i in value:
    dicts.append(tuple(i.split()))
print (dicts[49688])


vectorizer = MultiLabelBinarizer(sparse_output=True).fit(dicts)

#prod_dict = {value:key  for (value, key) in zip(value, key)}
#print (list(vectorizer.classes_))
#print (key[49688])
#print (value[49688])
#vectorizer = TfidfVectorizer(vocabulary = prod_dict)
#vectorizer = CountVectorizer(vocabulary = prod_dict)
train_target = df_train['products_list'].values

print(train_target[1])
y_train = []
for i in train_target:
    y_train.append(tuple(i.split()))
print (y_train[0])
    

y_train = vectorizer.transform(y_train)

In [ ]:
display(df_train.iloc[900])

#custom round layer
from keras.layers import Layer
class Round(Layer):
    def __init__(self, **kwargs):
        super(Round, self).__init__(**kwargs)

    def get_output(self, train=False):
        X = self.get_input(train)
        return K.round(X)

    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(Round, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    
print (vectorizer.classes_[49688]) 

In [ ]:
y = y_train[:6000].todense()

y_eval = y[5950:]
y = y[:5950]

test = vectorizer.inverse_transform(y_train[1450:1500])
print(test[0])

In [ ]:
#input_layer = Input(shape=(keras_train.shape))
# create model
model = Sequential()
model.add(Dense(66,input_dim=X_train[0].shape, activation='relu'))
# Hidden layers
model.add(BatchNormalization())
model.add(Dense(124,activation='relu'))
model.add(Dense(386,activation='relu'))
model.add(Dense(512,activation='relu'))
#model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(512,activation='relu'))
model.add(Dense(700,activation='relu'))
model.add(Dropout(0.2))
#output layer
model.add(Dense(49689, activation='sigmoid'))

adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.99, epsilon=1e-03, decay=0.0001)
RMSprop = keras.optimizers.RMSprop(lr=0.001)
Nadam = keras.optimizers.Nadam(lr=0.001)
Adagrad = keras.optimizers.Adagrad(lr=0.001)
model.compile(loss='binary_crossentropy',optimizer= RMSprop, metrics = ['categorical_accuracy'])

keras_train = np.array(X_train[:5950])
keras_eval = np.array(X_train[5950:6000])

print (y_eval.shape)

In [ ]:
print('Training')
# since we are using stateful rnn tsteps can be set to 1
tsteps = 1
batch_size = 25
epochs = 25
# Fit the model

model.fit(keras_train, y, epochs=5, batch_size=900,verbose=2, validation_split = 0.05) 

score = model.evaluate(keras_eval, y_eval)

print('\nTest score:', score[0])
print('Test accuracy:', score[1])

#for i in range(epochs):
#    print('Epoch', i, '/', epochs)

    # Note that the last state for sample i in a batch will
    # be used as initial state for sample i in the next batch.
    # Thus we are simultaneously training on batch_size series with
    # lower resolution than the original series contained in cos.
    # Each of these series are offset by one step and can be
    # extracted with cos[i::batch_size].
#    model.fit(X_train, train_target,
#              batch_size=batch_size,
#              epochs=1,
#              verbose=1)
#    model.reset_states()


In [ ]:
print('Predicting')
predicted_output = model.predict(keras_eval)

predicted_output[predicted_output>=0.5] = 1
predicted_output[predicted_output<0.5] = 0

y_eval_pred = vectorizer.inverse_transform(predicted_output)

print('true eval set values {}'.format(test[0]))
print('pred eval set values {}\n'.format(y_eval_pred[0]))

print (multilabel_fscore(test,y_eval_pred))